In [141]:
def tokenize(the_list):
    for i in range(0,len(the_list)):
        #the_list[i] = the_list[i].replace('\n', ' ')
        the_list[i] = the_list[i].replace(';', ' ; ')
        the_list[i] = the_list[i].replace(',', ' , ')
        the_list[i] = the_list[i].replace('.', ' . ')
        the_list[i] = the_list[i].replace('"', '')
        the_list[i] = the_list[i].replace('«', '')
        the_list[i] = the_list[i].replace('»', '')
        the_list[i] = the_list[i].replace(')', ' ) ')
        the_list[i] = the_list[i].replace('(', ' ( ')
        the_list[i] = the_list[i].replace('  ', ' ')
        the_list[i] = the_list[i].replace('   ', ' ')
        the_list[i] = the_list[i].replace('    ', ' ')

## imports

In [2]:
import pandas as pd
import os
import copy
import re

## files reading

In [165]:
dir_path = 'Collection3'

In [166]:
file_pathes = []
for top, dirs, files in os.walk(dir_path):
    for nm in files:
        file_pathes.append(os.path.join(top, nm))

In [181]:
files = []
for i in range(0, len(file_pathes)):
    files.append(os.path.basename(file_pathes[i]))
len(files)

names = []
names.append(' ')
for file in files:
    name, ext = os.path.splitext(file)
    if name == names[-1]:
        del names[-1]
    names.append(name)
names = names[1:]
len(names)

1000

## data reading (loop over all files)

In [188]:
%time
data = []
for name in names:
    raw_data = []
    for line in open(dir_path+ '//'+ name + '.ann', 'r', encoding='utf-8'):
        raw_data.append(line)
        tags = []
        starts = []
        ends = []
        for i in range(0, len(raw_data)):
            s = raw_data[i]
            parts = s.split('\t') 
            s1 = parts[1]
            parties = s1.split(' ')
            tags.append(parties[0])
            starts.append(parties[1])
            ends.append(parties[2])
     
    content = []
    for line in open(dir_path+ '//'+ name + '.txt', 'r', encoding='utf-8'):
        content.append(line)
    #tokenize(content)
    content = ' '.join(content)
    #content = re.sub(r' +', ' ', content)
    

    tagged_content = copy.copy(content)
    
    list_content = list(tagged_content)

    for i in range(0, len(list_content)):
        if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
            list_content[i] = 'O'

    tagged_content = "".join(list_content)
    

    for i in range(0, len(tags)):
        for j in range(int(starts[i]), int(ends[i])):
            if content[j] == ' ':
                list_content[j] = ' '
            else:
                list_content[j] = tags[i]

    tagged_content2 = "".join(list_content)
    
    
    tagged_content2 = re.sub(r'PER+', 'P', tagged_content2)
    tagged_content2 = re.sub(r'P+', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'LOC+', 'L', tagged_content2)
    tagged_content2 = re.sub(r'L+', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'DAT+', 'D', tagged_content2)
    tagged_content2 = re.sub(r'D+', 'DAT', tagged_content2)
    tagged_content2 = re.sub(r'O+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'ORG+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'OO+', 'ORG', tagged_content2)

    
    #так как мы используем хорошие эмбеддинги, которые не рассматривают слово как неделимую штуку, мы можем не отделять запятые и прочее как отдельные токены
    #поэтому артефакты от значимых тегов - просто уберем
    tagged_content2 = re.sub(r'OPER', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'PERO', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'OLOC', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'LOCO', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'ODAT', 'DAT', tagged_content2)
    tagged_content2 = re.sub(r'DATO', 'DAT', tagged_content2)
    tagged_content2 = re.sub(r'OORG', 'ORG', tagged_content2)
    tagged_content2 = re.sub(r'ORGO', 'ORG', tagged_content2)


    tagged_content2 = re.sub(r'\n', '', tagged_content2)
    content = re.sub(r'\n', '', content)

    data.append([tagged_content2, content])

df = pd.DataFrame(data, columns=['tags', 'texts'])
df.to_csv('Collection3.csv', index = False)

Wall time: 0 ns


# NOT LOOP ## with output to look on data

In [4]:
raw_data = []
for line in open('001.ann', 'r', encoding='utf-8'):
    raw_data.append(line)
raw_data


['T1\tLOC 0 6\tРоссия\n',
 'T2\tLOC 50 53\tСША\n',
 'T3\tLOC 57 63\tГрузию\n',
 'T4\tLOC 87 93\tМОСКВА\n',
 'T5\tORG 103 114\tРИА Новости\n',
 'T6\tLOC 116 122\tРоссия\n',
 'T7\tLOC 141 144\tСША\n',
 'T8\tLOC 161 168\tТбилиси\n',
 'T9\tLOC 301 307\tРоссии\n',
 'T10\tPER 308 324\tГригорий Карасин\n',
 'T11\tLOC 383 386\tСША\n',
 'T12\tPER 387 402\tДэниэлом Фридом\n',
 'T13\tLOC 505 517\tЮжной Осетии\n',
 'T14\tLOC 703 709\tРоссия\n',
 'T15\tLOC 723 730\tТбилиси\n',
 'T16\tLOC 815 825\tВашингтона\n',
 'T17\tORG 838 841\tМИД\n',
 'T18\tLOC 842 848\tРоссии\n']

In [140]:
tags = []
starts = []
ends = []
for i in range(0, len(raw_data)):
    s = raw_data[i]
    parts = s.split('\t') 
    s1 = parts[1]
    parties = s1.split(' ')
    tags.append(parties[0])
    starts.append(parties[1])
    ends.append(parties[2])

In [142]:
content = []
for line in open('001.txt', 'r', encoding='utf-8'):
    content.append(line)
#tokenize(content)
content = ' '.join(content)
#content = re.sub(r' +', ' ', content)
content

'Россия рассчитывает на конструктивное воздействие США на Грузию\n \n 04/08/2008 12:08\n \n МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\n \n "С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\n \n "Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. '

In [143]:
tagged_content = copy.copy(content)
tagged_content

'Россия рассчитывает на конструктивное воздействие США на Грузию\n \n 04/08/2008 12:08\n \n МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\n \n "С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\n \n "Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. '

In [144]:
len(tagged_content) == len(content)

True

In [145]:
list_content = list(tagged_content)

for i in range(0, len(list_content)):
    if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
        list_content[i] = 'O'
        
tagged_content = "".join(list_content)

In [146]:
len(tagged_content) == len(content)

True

In [147]:
#list_content = list(tagged_content[0])

for i in range(0, len(tags)):
    for j in range(int(starts[i]), int(ends[i])):
        if content[j] == ' ':
            list_content[j] = ' '
        else:
            list_content[j] = tags[i]
        
tagged_content2 = "".join(list_content)

In [148]:
tagged_content2

'LOCLOCLOCLOCLOCLOC OOOOOOOOOOOO OO OOOOOOOOOOOOOO OOOOOOOOOOO LOCLOCLOC OO LOCLOCLOCLOCLOCLOC\n \n OOOOOOOOOO OOOOO\n \n LOCLOCLOCLOCLOCLOCO O OOO O ORGORGORG ORGORGORGORGORGORGORGO LOCLOCLOCLOCLOCLOC OOOOOOOOOOOOO OOO LOCLOCLOC OOOOOOOOOOOO OO LOCLOCLOCLOCLOCLOCLOC O OOOOO O OOOOOOOOOOO OOOOOOOO O OOOO OOOOOOOOOOOOOOOOOOO OOOOOOOOOO OO OOOO OOOOOOOOOOOOOOO O OOOOOOOOOOO OOOOOOOO OOOOOOOOOOO OOO LOCLOCLOCLOCLOCLOC PERPERPERPERPERPERPERPER PERPERPERPERPERPERPER OOOOOO O OOOOOOOOOO OOOOOOOOO O OOOOOOOOOOOO OOOOOOOOOOOO LOCLOCLOC PERPERPERPERPERPERPERPER PERPERPERPERPERPERO\n \n OO OOOOOOOOOO OOOOOOO OOOOOOOO OOOOOOOO OOOOOOOOOOOOO O OOOOO O OOOOO OOOOOO OOOOOOOOOOOOO OOOOOO LOCLOCLOCLOCLOC LOCLOCLOCLOCLOCLOCO OOOOOOOOOOOOOOOO OOOOOOOOOO OOOOOOOOOO OOOOOOO OO OOOOOOOOOOO OOOOO OOOOOOOOOOO OOO O OOOOOOOO OOOOOOOOOOOOOO OOOOOOOOOOOOOO OOOOOOOOOOOOOOOO OOOOOOOOOOOO O OOOOOOOOO O OOOOOOOOOO\n \n OLOCLOCLOCLOCLOCLOC OOO OOOOOOOO LOCLOCLOCLOCLOCLOCLOC O OOOOOOOOOOOOO OOOOO O OOOOOOOOOOOO OOOOO

In [149]:
tagged_content2 = re.sub(r'PER+', 'P', tagged_content2)
tagged_content2 = re.sub(r'P+', 'PER', tagged_content2)
tagged_content2 = re.sub(r'LOC+', 'L', tagged_content2)
tagged_content2 = re.sub(r'L+', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'DAT+', 'D', tagged_content2)
tagged_content2 = re.sub(r'D+', 'DAT', tagged_content2)
tagged_content2 = re.sub(r'O+', 'O', tagged_content2)
tagged_content2 = re.sub(r'ORG+', 'O', tagged_content2)
tagged_content2 = re.sub(r'OO+', 'ORG', tagged_content2)
tagged_content2

'LOC O O O O LOC O LOC\n \n O O\n \n LOCO O O O ORG ORG LOC O O LOC O O LOC O O O O O O O O O O O O O O O O O LOC PER PER O O O O O O O LOC PER PERO\n \n O O O O O O O O O O O O O LOC LOCO O O O O O O O O O O O O O O O O O O O\n \n OLOC O O LOC O O O O O O O O O O O LOCO O O ORG LOCO '

In [154]:
#так как мы используем хорошие эмбеддинги, которые не рассматривают слово как неделимую штуку, мы можем не отделять запятые и прочее как отдельные токены
#поэтому артефакты от значимых тегов - просто уберем
tagged_content2 = re.sub(r'OPER', 'PER', tagged_content2)
tagged_content2 = re.sub(r'PERO', 'PER', tagged_content2)
tagged_content2 = re.sub(r'OLOC', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'LOCO', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'ODAT', 'DAT', tagged_content2)
tagged_content2 = re.sub(r'DATO', 'DAT', tagged_content2)
tagged_content2 = re.sub(r'OORG', 'ORG', tagged_content2)
tagged_content2 = re.sub(r'ORGO', 'ORG', tagged_content2)

In [160]:
tagged_content2 = re.sub(r'\n', '', tagged_content2)
content = re.sub(r'\n', '', content)

In [161]:
content

'Россия рассчитывает на конструктивное воздействие США на Грузию  04/08/2008 12:08  МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.  "С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.  "Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. '

## if you need split strings, and check a concurrency of tokens (words) and tags

In [162]:
words = content.split(' ')
tags = tagged_content2.split(' ')

In [163]:
len(tags)==len(words)

True

In [164]:
for i in range(len(words)):
    print('word ', words[i])
    print('tag ', tags[i])
    print('\n')

word  Россия
tag  LOC


word  рассчитывает
tag  O


word  на
tag  O


word  конструктивное
tag  O


word  воздействие
tag  O


word  США
tag  LOC


word  на
tag  O


word  Грузию
tag  LOC


word  
tag  


word  04/08/2008
tag  O


word  12:08
tag  O


word  
tag  


word  МОСКВА,
tag  LOC


word  4
tag  O


word  авг
tag  O


word  -
tag  O


word  РИА
tag  ORG


word  Новости.
tag  ORG


word  Россия
tag  LOC


word  рассчитывает,
tag  O


word  что
tag  O


word  США
tag  LOC


word  воздействуют
tag  O


word  на
tag  O


word  Тбилиси
tag  LOC


word  в
tag  O


word  связи
tag  O


word  с
tag  O


word  обострением
tag  O


word  ситуации
tag  O


word  в
tag  O


word  зоне
tag  O


word  грузино-осетинского
tag  O


word  конфликта.
tag  O


word  Об
tag  O


word  этом
tag  O


word  статс-секретарь
tag  O


word  -
tag  O


word  заместитель
tag  O


word  министра
tag  O


word  иностранных
tag  O


word  дел
tag  O


word  России
tag  LOC


word  Григорий
tag  PER


word  К